In [1]:
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data = pd.read_csv('/kaggle/input/ids-dataset/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv')
data = data.fillna(data.mean())
data.head()

/tmp/ipykernel_20/940602734.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data = data.fillna(data.mean())


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,80,38308,1,1,6,6,6,6,6.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,389,479,11,5,172,326,79,0,15.636364,31.449238,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,88,1095,10,6,3150,3150,1575,0,315.000000,632.561635,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,389,15206,17,12,3452,6660,1313,0,203.058823,425.778474,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,88,1092,9,6,3150,3152,1575,0,350.000000,694.509719,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [3]:
# Ensuite, nous devons séparer les données en features et labels

X = data.iloc[:, :-1]
y = data.iloc[:, -1]

# utiliser le PCA pour réduire la dimensionnalité des features
#pca = PCA(n_components=35)
#X = pca.fit_transform(X)

In [4]:
pca = PCA(n_components=35)
int_col = [col for col in X.columns.tolist() if X[col].dtypes == 'int64']
X = pca.fit_transform(X[int_col])

#### NB : Pour aller vite, je vais juste utiliser une portion de la donnée pour le trainning et l'évaluation

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X_test, y_test, test_size=0.2, random_state=123)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((27708, 35), (6928, 35), (27708,), (6928,))

In [6]:
# Nous pouvons maintenant entrainer notre modèle SVM

svm = SVC(kernel='rbf', gamma='scale')
svm.fit(X_train, y_train)

SVC()

In [7]:
# nous pouvons l'utiliser pour prédire les labels sur les données de test et calculer la précision

y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.8446882217090069


In [8]:
output = pd.DataFrame(y_pred)
output.to_csv("output.csv", index=False)
output

,0
0,BENIGN
1,BENIGN
2,BENIGN
3,BENIGN
4,BENIGN
...,...
6923,DoS Hulk
6924,BENIGN
6925,BENIGN
6926,DoS GoldenEye
